# Part 1

## An Introductory example

Source: https://lectures.quantecon.org/jl/julia_by_example.html

Iniciamos com um plot bestinha de ruído branco:

In [12]:
#using Plots #equivalente ao library(...) do R, tem o mesmo efeito de colocar tudão no global scope. Poderia ter usado import Plots: plot para apenas a função como Python
import Plots: plot
ts_length = 100
epsilon_values = randn(ts_length)
plot(epsilon_values, color = "blue")

O vetor de ruídos aleatórios acima cria um vetor.

In [13]:
typeof(epsilon_values)

Array{Float64,1}

Que é um vetorzão dos valores aleatórios 

In [14]:
epsilon_values

100-element Array{Float64,1}:
 -0.289603 
  0.356813 
 -1.05734  
 -0.148724 
 -0.474733 
  0.449271 
  0.910569 
  0.910735 
 -0.0989941
 -1.03228  
  1.51223  
  1.30684  
  1.01826  
  ⋮        
 -0.0216587
 -0.0419765
 -1.25999  
 -2.0325   
 -0.876229 
 -1.08427  
 -0.561685 
  0.135765 
  0.609344 
  2.80297  
 -1.06653  
 -0.576519 

Assim como R, podemos colocar qualquer coisa em um vetor

In [15]:
x = [10,"etc",false]
typeof(x[1]) #Indexing como no R
typeof(x[2])
typeof(x[3])

Bool

Funções com ! modificam (ou 'mutam' no léxico aqui) o argumento. Então

In [16]:
pop!(x)
x

2-element Array{Any,1}:
 10     
   "etc"

Da mesma forma que `pop!`, a função `push!` modifica o argumento, mas faz o inverso e adiciona um elemento ao final desse array

In [17]:
push!(x, "Oi")
x

3-element Array{Any,1}:
 10     
   "etc"
   "Oi" 

Os controles de fluxo apresentam sintaxes bem próximos aos de python. Por exemplo, os `for` loops

In [21]:
ts_length = 100
epsilon_values = Array{Float64}(ts_length)
for i in 1:ts_length
    epsilon_values[i] = randn()
end
plot(epsilon_values,color = "blue")

Assim como R e Python, podemos iterar sobre iteradores, vetores servindo como indices

In [23]:
palavras = ["primeira","segunda"]
for pala in palavras
    println("Essa é a $pala palavra")
end

Essa é a primeira palavra
Essa é a segunda palavra


Apesar de ver pouco uso pra mim, posso usar um while também

In [24]:
ts_length = 100
epsilon_values = Array{Float64}(ts_length)
i = 1
while i <= ts_length
    epsilon_values[i] = randn()
    i+=1
end
plot(epsilon_values,color = "blue")

Funções estão bem flexíveis no Julia, com a definição usual, uma expressa (usando `f_nome`), uma para tuplas, lexical scoping e mais. Um exemplo da definição básica para gerar os dados é dada por 

In [25]:
function gera_dados(n)
    epislon_values = Array{Float64}(n)
    for i = 1:n
        epsilon_values[i] = randn()
    end
    return epsilon_values
end

ts_length = 100
data = gera_dados(ts_length)
plot(data, color = "blue")

Usemos algo mais estatístico. Para isso, usaremos o pacote `Distributions`

In [26]:
Pkg.add("Distributions")

INFO: Package Distributions is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Distributions
INFO: Use `Pkg.update()` to get the latest versions of your packages


Façamos um histograma

In [28]:
using Distributions

function plot_hist(distribution,n) #note que uma das entradas é um tipo do Distributions, 'distribution'
    epsilon_values = rand(distribution, n) #n amostras de uma distribuição especificada! Que bonitinho!
    histogram(epsilon_values)
end

lp = Laplace() #Objeto do tipo 'distribuição' <3

plot_hist(lp,1000)

Extremamente simpático existir um objeto do tipo "distribuição" e todos os métodos parecem estar definidos sobre objetos dessa classe, como amostras (`rand()`) ou histogramas (`histogram`). (Nota: Preciso achar a documentação desses pacotes para ver os métodos disponíveis). 

Em tempo: Esse super 'overloading' como no R se chama **multiple dispatch**, e é um preceito fundamental no design da Julia.

### Exercícios

#### Exercício 1

Podemos calcular o fatorial $n!$ com um for utilizando o código

In [29]:
function factorial2(n)
    res = 1
    for i in 1:n
        res*=i
    end
    return res
end

factorial2(5)

120

#### Exercício 2

Para obter uma função que gere uma amostra da $Binomial(N,p)$ apenas usando o `rand()`

In [37]:
function binomial_rv(n,p)
    amostras = Array{Float64}(n)
    for i in 1:n
        U = rand()
        if U <=p
            amostras[i] = 1
        else
            amostras[i] = 0
        end
    end
    return sum(amostras)
end

binomial_rv(10,0.3)

4.0

#### Exercício 3 

Clássico de aproximar $\pi$ utilizando Monte Carlo

In [48]:
function MC(n)
    U = rand(n,2) #realizacoes das coordenadas
    Area = 0
    for i in 1:n
        if sqrt(U[i,1]^2+U[i,2]^2) <=1
            Area +=1/n
        else
            Area +=0
        end
    end
    4*Area #Pois como sao duas U(0,1) temos apenas 1/4 da área
end

MC(100000)

3.14199999999624

#### Exercício 4 

In [17]:
using Distributions
x = Binomial(1,0.5)
res = rand(x,10)
dolar = 0 #pagamento
for i in 2:(length(res)-1)
    if res[i-1]==res[i]==res[i+1]==1
        dolar = 1
    end
end
dolar

1

Agora vi que o exercício pede apenas `rand()`. Ok.

In [20]:
res = Array{Int64}(10)
for i in 1:10
    if(rand() <= 0.5)
        res[i] = 1
    else
        res[i] = 0
    end
end
dolar = 0 #pagamento
for i in 2:(length(res)-1)
    if res[i-1]==res[i]==res[i+1]==1
        dolar = 1
    end
end
dolar

0

#### Exercício 5 

Considere a série temporal $x_{t+1} = \alpha x_t+\epsilon_{t+1} $, com $x_0 = 0$

In [56]:
using Distributions
using Plots
T = 200
α = 0.9
N = Normal(0,1)
x = Array{Float64}(T)
x[1] = rand(N,1)[1]
for i in 2:T
    x[i] = α*x[i-1] + rand(N,1)[1]
end
println(x)
plot(x,color = "red")


[-0.63306, -1.59451, -1.61367, -2.65212, -1.89319, -1.74882, -2.69726, -2.34733, -2.23035, -2.56364, -2.52772, -2.27411, -2.83782, -3.26042, -2.32893, -3.09879, -2.27334, -0.8996, -1.07242, 0.153524, -0.400782, 0.0460009, 0.434175, 0.743563, 0.339677, -0.114308, -0.262333, -0.352092, -0.0900637, 0.452145, -0.999151, -0.839302, -1.41192, -0.912993, -0.311098, -0.675282, -1.44534, -1.89594, 0.465073, 1.9989, 1.98532, 1.84882, 0.970326, 0.100799, -0.683279, 0.369912, -0.278636, 0.249603, 0.832588, 0.00737486, 1.01958, -0.273003, -0.286978, 0.436939, 0.41778, -0.323217, -1.20539, -1.76157, -1.6865, -1.39604, -1.40219, -1.42175, -1.99789, -1.81046, -2.09958, -2.01788, -2.28829, -2.50154, -2.85534, -4.62678, -5.56421, -6.31178, -4.84806, -3.78353, -3.91955, -3.94356, -3.54715, -2.85871, -0.0594064, 0.0157831, -1.23784, -2.53167, -3.90857, -3.86437, -2.52857, -1.04008, -0.919217, -1.39582, -1.15833, -2.01125, -1.70554, -2.63164, -2.06536, -3.40049, -4.80928, -3.21822, -2.19801, -1.85767, -2.0

#### Exercício 6

In [4]:
using Distributions
using Plots
function simTS(α, T=200)
    T = 200
    N = Normal(0,1)
    x = Array{Float64}(T)
    x[1] = rand(N,1)[1]
    for i in 2:T
        x[i] = α*x[i-1] + rand(N,1)[1]
    end
    return x
end

alfas = [0.00,0.80,0.98]
series = []
labels = []

for par in alfas
    push!(series,simTS(par))
    push!(labels,par)
end
plot(series,labels = label=reshape(labels,1,length(labels)))




## Julia Essentials

Julia tem os tipos básicos d